# Results combination and analysis

In [15]:
%%html
<style>
table {float:left}
</style>

In [16]:
%load_ext kedro.ipython
%reload_kedro
%load_ext autoreload
%autoreload 2
%config IPCompleter.use_jedi=False

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[06/28/23 15:10:53] INFO     Resolved project path as:                                              ]8;id=913338;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=562433;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/28/23 15:10:54] INFO     Kedro project Project RDI Cape Town                                    ]8;id=706500;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=307545;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=386260;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=560012;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [17]:
import csv
import glob
import io
import logging
import os
import subprocess
from datetime import datetime

import plotly.express as px
import plotly.io as pio
import requests

pio.renderers.default = "iframe"

import re

import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import yaml
from GPSOdyssey import Kepler
from IPython.core.interactiveshell import InteractiveShell

# Ipython configs
from IPython.display import HTML, display
from shapely.geometry import LineString

display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

## Find partition keys

In [18]:
df_producer_geo_all = catalog.load("syn_pop_scenarios_local_20220601")
waste_all = catalog.load("waste_gen_scenarios_local_20220601")
results_all = catalog.load("resource_summary_table_arcs_only_20220601_decentral")
keys = list(df_producer_geo_all)
keys = [x for x in keys if x[0] != "."]

[06/28/23 15:10:57] INFO     Loading data from 'syn_pop_scenarios_local_20220601'               ]8;id=922436;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=893869;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from 'waste_gen_scenarios_local_20220601'             ]8;id=654196;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=232800;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from                                                  ]8;id=514047;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=876868;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             'resource_summary_table_arcs_only_20220601_decentral'                                 
                             (PartitionedDataSet)...                                                               

## Extract and summaries

In [19]:
i = 0

In [20]:
all_results = []
for key in keys:
    results = results_all[key]()
    results = results.loc[results["Unnamed: 0"] == "Total"]
    results["scenario"] = key
    all_results.append(results)
all_results = pd.concat(all_results)
all_results = all_results.drop(columns=["Unnamed: 0", "Collection day"])
all_results[
    ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
] = all_results[
    ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
].fillna(
    0
)
all_results = all_results.assign(
    **{"Bins collected": all_results["Units served"].astype(int)}
)

## Store results

In [22]:
%reload_kedro
catalog.save("resource_scenario_results_20220601_decentral_arcs_only", all_results)

[06/28/23 15:11:30] INFO     Resolved project path as:                                              ]8;id=14906;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=441466;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/28/23 15:11:30] INFO     Kedro project Project RDI Cape Town                                    ]8;id=209163;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=426583;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=175479;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=716058;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

                    INFO     Saving data to                                                     ]8;id=931841;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=53648;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#384\384]8;;\
                             'resource_scenario_results_20220601_decentral_arcs_only'                              
                             (CSVDataSet)...                                                                       

In [13]:
all_results

,Vehicle,Route,Offloads,Bins collected,Units served,Demand collected (ton),Time collecting (h),Time travelling (h),Time at treatment facility (h),Route duration (h),Traveling distance (km),Collecting distance (km),Total route distance (km),Electrical consumption (kWh),Tons disposed at @ Offload 2,Tons disposed at @ Offload 3,scenario
10,2,10,48,40971,40971,432.33,78.51,10.89,12.00,101.47,492.81,322.90,815.71,1885.67,147.06,285.28,households_001
10,2,10,47,40949,40949,432.51,77.91,11.74,11.75,101.42,529.16,306.98,836.14,1897.72,163.53,268.99,households_002
10,2,10,48,41009,41009,433.33,78.07,11.19,12.00,101.30,505.51,308.59,814.10,1873.77,150.85,282.48,households_003
10,2,10,46,40992,40992,433.33,78.02,11.26,11.50,100.84,509.16,308.13,817.28,1877.23,152.68,280.66,households_004
10,2,10,47,40980,40980,435.89,78.00,11.63,11.75,101.42,524.89,308.12,833.01,1899.94,182.50,253.39,households_005
10,2,10,48,40953,40953,433.58,77.90,11.11,12.00,101.05,502.06,306.27,808.32,1865.37,161.00,272.59,households_006
10,2,10,48,40595,40595,435.64,77.92,11.27,12.00,101.22,508.62,307.85,816.48,1879.51,178.47,257.18,households_007
10,2,10,48,40974,40974,435.55,77.96,11.12,12.00,101.12,502.33,307.10,809.43,1870.33,164.34,271.22,households_008
10,2,10,47,40950,40950,434.58,77.94,11.68,11.75,101.39,526.50,307.74,834.23,1899.14,177.77,256.80,households_009
10,2,10,48,40983,40983,434.72,78.03,11.48,12.00,101.52,517.07,308.61,825.68,1889.77,174.32,260.38,households_010


## Early graphs

In [14]:
fig = px.histogram(all_results, x="Demand collected (ton)")
fig

fig = px.histogram(all_results, x="Units served")
fig

In [8]:
fig = px.histogram(all_results, x="Route duration (h)")
fig

In [9]:
fig = px.histogram(all_results, x="Total route distance (km)")
fig

In [10]:
fig = px.scatter(all_results, x="Units served", y="Route duration (h)")
fig
fig = px.scatter(all_results, x="Units served", y="Total route distance (km)")
fig

In [12]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Route duration (h)")
fig
fig = px.scatter(all_results, x="Demand collected (ton)", y="Total route distance (km)")
fig

In [13]:
fig = px.scatter(all_results, x="Units served", y="Demand collected (ton)")
fig

In [14]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Total route distance (km)", color="Units served")
fig

In [15]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Route duration (h)", color="Units served")
fig

In [16]:
from pandas_profiling import ProfileReport

2022-06-03 00:14:15,477 - visions.backends - INFO - Pandas backend loaded 1.4.2
2022-06-03 00:14:15,489 - visions.backends - INFO - Numpy backend loaded 1.22.3
2022-06-03 00:14:15,491 - visions.backends - INFO - Pyspark backend NOT loaded
2022-06-03 00:14:15,492 - visions.backends - INFO - Python backend loaded


In [17]:
profile = ProfileReport(all_results, title="Waste generation scenario analysis - 20220601")

In [18]:
profile.to_file("../data/08_reporting/Waste generation scenario analysis - 20220601.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/missingno/missingno.py:72: MatplotlibDeprecationWarning:

The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.



Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]